# Download OSM Data with OSMNX

In [ ]:
import osmnx as ox
import geopandas as gpd
from sklearn.neighbors import NearestNeighbors
import numpy as np

1. Load Data Graz Polygon

In [2]:
place_name = "Graz, Austria"
graz = ox.geocode_to_gdf(place_name)

graz.to_file("graz.shp")

/tmp/ipykernel_62/4235810593.py:6: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  graz.to_file("graz.shp")


2. Load Data Geidorf Polygon

In [ ]:
place_name = "Geidorf, Graz, Austria"
geidorf = ox.geocode_to_gdf(place_name)

geidorf.to_file("geidorf_bezirk.shp")

3. Load Data Buildings Graz

In [ ]:
place_name = "Graz, Austria"

building_tags = {'building': True}
buildings = ox.features_from_place(place_name, building_tags)
# only polygon data
buildings = buildings[buildings.geometry.type == 'Polygon']
buildings = buildings[buildings.geometry.apply(lambda geom: geom.is_valid)]

buildings = gpd.GeoDataFrame(buildings, crs="EPSG:4326")

buildings = buildings[['geometry']]

buildings.to_file("buildings.shp")

/tmp/ipykernel_469/342192505.py:19: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  buildings.to_file("buildings.shp")


4. Load Data Supermarkets Graz

In [ ]:
place_name = "Graz, Austria"

supermarket_tags = {
    'shop': ['supermarket', 'grocery']
}

supermarkets = ox.features_from_place(place_name, supermarket_tags)
# only polygon data
supermarkets = supermarkets[supermarkets.geometry.type == "Polygon"]
supermarkets = supermarkets[supermarkets.geometry.apply(lambda geom: geom.is_valid)]

supermarkets = gpd.GeoDataFrame(supermarkets, crs="EPSG:4326")

supermarkets = supermarkets[['geometry','brand']]

supermarkets.to_file("supermarkets.shp")

# Data processing and nearest neighbor (Buildings and Supermarkets)

In [ ]:
buildings = buildings.to_crs(31256)
supermarkets = supermarkets.to_crs(31256)
# find centerpoint for each polygon
buildings['centroid'] = buildings.geometry.centroid
supermarkets['centroid'] = supermarkets.geometry.centroid

building_coords = np.array([(geom.x, geom.y) for geom in buildings['centroid']])
supermarket_coords = np.array([(geom.x, geom.y) for geom in supermarkets['centroid']])
# nearest supermarket for each building
nbrs = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(supermarket_coords)
distances, indices = nbrs.kneighbors(building_coords)

buildings['supermarket_index'] = indices
buildings['distance_to_supermarket'] = distances

buildings = buildings.to_crs(4326)
supermarkets = supermarkets.to_crs(4326)
buildings['centroid'] = buildings['centroid'].to_crs(4326)
supermarkets['centroid'] = supermarkets['centroid'].to_crs(4326)
# join buildings with supermarkets
buildings['supermarket_index'] = buildings['supermarket_index'].astype(int)
supermarkets = supermarkets.reset_index()
buildings = buildings.join(supermarkets, on='supermarket_index', rsuffix='_supermarket')

buildings["lon_geb"] = buildings.geometry.apply(lambda geom: geom.x if geom.geom_type == 'Point' else geom.centroid.x)
buildings["lat_geb"] = buildings.geometry.apply(lambda geom: geom.y if geom.geom_type == 'Point' else geom.centroid.y)

buildings.to_parquet("buildings.parquet")
supermarkets.to_parquet("supermarkets.parquet")